# Matrix Multiplication 
This notebook compares time of matrix multiplication on GPU and on CPU using <a href='https://www.tensorflow.org/'>TensorFlow</a>

In [28]:
import tensorflow as tf
import numpy as np
import time

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)


%matplotlib inline 

## Interactive Plot
Results will be plot using <a href='https://plot.ly/'>Plotly</a> visualization library.

In [29]:
def show_interactive(sizes, gpu, cpu, filename):
    gpu_fig = go.Scatter(
        x = sizes,
        y = gpu,
        mode = 'lines+markers',
        name = 'gpu'
    )
    
    cpu_fig = go.Scatter(
        x = sizes,
        y = cpu,
        mode = 'lines+markers',
        name = 'cpu'
    )

    data = [gpu_fig, cpu_fig]

    layout = dict(title = 'Comparision of matrix multiplication time between CPU and GPU',
              xaxis = dict(title = 'Square matrix size'),
              yaxis = dict(title = 'Time (seconds)'),
              )
    fig = dict(data=data, layout=layout)

    iplot(fig, filename=filename)

## Measure single execution
This function executes given tensorflow expression and measures it's time of execution

In [30]:
def measure_time(expression):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    with tf.Session(config=config) as sess:
        start_time = time.time()
        result = sess.run(expression)
        time_taken = time.time() - start_time
    
    return time_taken

## Testing
For given list of sizes, perform matrix multiplication and collects results for GPU and CPU

In [31]:
def test(sizes):
    devices = ['/gpu:0','/cpu:0']
    results = []
    for device in devices:
        for size in sizes:
            shape = (size,size)
            data_type = tf.float32
            
            with tf.device(device):
                a = tf.random_normal(shape=shape, dtype=data_type)
                b = tf.random_normal(shape=shape, dtype=data_type)
                c = tf.matmul(a,b)
            
            #print 'DEVICE',device,'MATRIX_SHAPE',shape, 'EXECUTION_TIME',measure_time(c)
            results.append({'DEVICE':device,'MATRIX_SHAPE':shape, 'EXECUTION_TIME':measure_time(c)})
    return results

In [35]:
sizes = [100, 500, 1000, 3000, 5000, 6500, 8000, 10000]
results = test(sizes)

In [36]:
cpu = [x['EXECUTION_TIME'] for x in results if x['DEVICE']=='/cpu:0']
gpu = [x['EXECUTION_TIME'] for x in results if x['DEVICE']=='/gpu:0']

show_interactive(sizes, gpu, cpu, 'cpu_vs_gpu')